In [ ]:
import cv2
import numpy as np
import torch
from torchvision.models import detection
from sort import *
from torchvision import transforms

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = detection.fasterrcnn_resnet50_fpn(pretrained=True, progress=True, num_classes=91, pretrained_backbone=True).to(device)
model.eval()

In [ ]:
classes = []
with open("./coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [ ]:
cap = cv2.VideoCapture('Video2.mp4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out1 = cv2.VideoWriter("./output_video.mp4", fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))
tracker = Sort()
Count_Num = 0
Count_Cars = []

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    tensor = torch.from_numpy(frame).permute(2, 0, 1).float().to(device)
    tensor /= 255.0
    tensor = tensor.unsqueeze(0)
    with torch.no_grad():
        predictions = model(tensor)[0]
        
    boxes = predictions['boxes'].cpu().numpy()
    scores = predictions['scores'].cpu().numpy()
    cars_idx = np.where(predictions['labels'].cpu().numpy() == 3)[0]
    class_ids = []
    new_boxes = []
    confidences = []
    for idx in cars_idx:
        if scores[idx] > 0.5:
            box = boxes[idx].astype(np.int32)
            class_ids.append(2)
            confidences.append(float(scores[idx]))
            new_boxes.append([box[0], box[1], box[2]-box[0], box[3]-box[1]])
            
    indices = cv2.dnn.NMSBoxes(new_boxes, confidences, 0.5, 0.4)
    dets = []
    for i in indices:
        box = new_boxes[i]
        width = box[2]
        height = box[3]
        left = box[0]
        top = box[1]

        cv2.rectangle(frame, (left, top), (left+width, top+height), (0, 255, 0), 2)
        dets.append([left, top, left+width, top+height, confidences[
        for track in tracks:I]])

    dets = np.array(de
            bbox = track.astype(np.int32)
            x1, y1, x2, y2, track_id = bbox
            if track_id not in Count_Cars:
                Count_Cars.append(track_id)
                Count_Num += 1
    print(Count_Num)
    out1.write(frame)

In [ ]:
cap.release()
out1.release()
print(f"Total car count: {Count_Num}")ts)
    if dets.shape[0] > 0:
        tracks = tracker.update(dets)